In [95]:
using TimeSeries
using Pkg
using DataFrames,DataFramesMeta
using Dates
using Statistics
using Flux
using Flux: @epochs
using Plots
using Hyperopt

In [2]:
ENV["PYTHON"] = "/Users/kahingleung/PycharmProjects/mylightning/venv/bin/python3.8"
Pkg.build("PyCall")

   Building Conda ─→ `~/.julia/packages/Conda/x5ml4/deps/build.log`
   Building PyCall → `~/.julia/packages/PyCall/tqyST/deps/build.log`


In [3]:
using PyCall
function getStock(sym,period)
    yf = pyimport("yfinance")
    #sym = "gbtc"
    #sym = "2840.HK"
    ticker = yf.Ticker(sym)
    etf = ticker.history(period=period)

    function pd_to_df(df_pd)
        df= DataFrame()
        for col in df_pd.columns
            df[!, col] = getproperty(df_pd, col).values
        end
        df[!,:Date] = collect(df_pd[:index])
        return df
    end
    etf = pd_to_df(etf)

    transform!(etf, :Date => ( x -> Date.(Dates.year.(x),Dates.month.(x),Dates.day.(x)) ) => :ondate)
    for c in ["High","Low","Open","Close","Stock Splits"]
        transform!(etf,c => (x -> Float32.(x) )=> c)
    end
    return select(etf,[:Open,:Close,:High,:Low,:Volume,:ondate])
end

getStock (generic function with 1 method)

In [4]:
function toPL(PL_ts)
    function f_PL(values)
        open = values[1]
        close = values[2]
        if close - open > 0
            pl = 1
        else
            pl = 0
        end
        [pl,0]
    end     
    PL = TimeSeries.rename(TimeSeries.map((timestamp, values) -> (timestamp, f_PL(values)),PL_ts)[:Open],Symbol("PL") )
    PL = TimeSeries.map((timestamp, values) -> (timestamp, Int(values)),PL)
end

toPL (generic function with 1 method)

In [5]:
function toRSI(price_ts,loopback)
    pct = percentchange(price_ts)
    upidx = findall(pct["Close"] .> 0)
    downidx = findall(pct["Close"] .< 0)
    up = map( (timestamp, values) -> (timestamp, if values < 0 ; 0 ; else ;values ;end), pct)
    down = map( (timestamp, values) -> (timestamp, if values < 0 ; abs(values) ; else ;0 ;end), pct)
    up_roll = moving(mean, up, loopback)
    down_roll = moving(mean, down, loopback)
    function f_rsi(values)
        x = values[1]
        y = values[2]
        rsi = 100 - (100 / (1 + x/y) )
        [rsi,0]
    end
    updown = TimeSeries.rename(TimeSeries.merge(up_roll,down_roll), [:up,:down])
    rsi_ts = TimeSeries.rename(TimeSeries.map((timestamp, values) -> (timestamp, f_rsi(values)), updown)[:up],Symbol("RSI-",loopback))
    return rsi_ts
end

toRSI (generic function with 1 method)

In [49]:
function toORSI(price_ts,day0,day1)
    #pct = percentchange(price_ts)
    result_rsi = price_ts
    result_rsima = price_ts
    result_orsi = price_ts
    for i in day0:day1
        rsi_ts = toRSI(price_ts,i)
        result_rsi = merge(result_rsi,rsi_ts,method=:inner)
        for j in day0:day1
            rsi_ma = TimeSeries.rename(moving(mean,rsi_ts,j),Symbol("RSIMA-",i,"-",j))
            orsi = TimeSeries.rename(rsi_ts .- rsi_ma , Symbol("ORSI-",i,"-",j) )
            result_orsi = merge(result_orsi,orsi,method=:inner)
        end
    end
    #println(colnames(result_orsi))
    return result_orsi
end

toORSI (generic function with 1 method)

In [102]:
function toMatrixORSI(data,featureIdx,targetIdx)
    println(colnames(data)[1:5])
    #featureIdx=3
    data = values(data)
    X = data[:,featureIdx:end]
    #b = maximum(X)
    #a = minimum(X)
    #println(a,"-",b)
    #X = ( b .- X ) ./ (b-a+1)
    print(size(X))
    N = size(X)[1]
    k = Int(sqrt(size(X)[2]))
    println(k)
    println(X[1,1:4])
    println(X[2,1:4])
    println(X[2,k+1:k+4])
    X = transpose(X)
    X = reshape(X,(k,k,N))
    println(X[1:4,1,1])
    println(X[1:4,1,2])
    println(X[1:4,2,2])
    #targetIdx=1
    Y = data[:,targetIdx]
    return (X,Y)
end

toMatrixORSI (generic function with 2 methods)

In [101]:
function toReturn(price_ts)
    pct = TimeSeries.rename(percentchange(price_ts),[:return])
    pct = moving(mean,pct,5)
    return TimeSeries.map((timestamp, values) -> (timestamp, 100*values), pct)
end   

toReturn (generic function with 1 method)

In [98]:
function preprocessing(etf,day0,day1,mlclass)
    PL = select(etf,[:Open,:Close,:ondate])
    price = select(etf,[:Close,:ondate])
    PL_ts = TimeArray(PL,timestamp=:ondate)
    PL_ts = toPL(PL_ts)
    pct = TimeSeries.rename(percentchange(TimeArray(price,timestamp=:ondate)),[:return])
    price_ts = TimeArray(price,timestamp=:ondate)
    pct = toReturn(price_ts)
    #day0=5
    #day1=30
    result_orsi = toORSI(price_ts,day0,day1)
    tgt = merge(PL_ts,pct,method=:inner)
    data = merge(tgt,result_orsi,method=:inner)
    featureIdx = 4
    if mlclass == :reg
        tgt = 2
    else
        tgt = 1
    end
        
    (X,Y) = toMatrixORSI(data,featureIdx,tgt)
end

preprocessing (generic function with 2 methods)

In [65]:
function timeseriesDataset(X,Y,seqlen,mlclass)
    xtrain = Array{Float32,3}[]
    ytrain = Array{Float32,1}[]
    len = length(Y)
    for i in 1:len-seqlen-1
        mx = X[:,:,i:i+seqlen-1]
        xtrain = vcat(xtrain,[mx])
        if mlclass == :reg
            my = Y[i+seqlen]
        else
            my = Flux.onehot(Y[i+seqlen], [0,1])
        end
        ytrain = vcat(ytrain,[my])
    end
    return (xtrain,ytrain)
end
    

timeseriesDataset (generic function with 2 methods)

In [10]:
function build_model(Nh,seqlen, delta)
    a = floor(Int8,Nh)
    sz = ceil(Int8,delta/8)
    
    println(delta,"-",sz)
    return Chain(
    x -> Flux.unsqueeze(x,4),
    # First convolution, operating upon a 28x28 image
    Conv((2, 2), seqlen=>a, pad=(1,1), relu),
    MaxPool((2,2)),

    # Second convolution, operating upon a 14x14 image
    Conv((2, 2), a=>Nh, pad=(1,1), relu),
    MaxPool((2,2)),

    # Third convolution, operating upon a 7x7 image
    Conv((2, 2), Nh=>Nh, pad=(1,1), relu),
    MaxPool((2,2)),

    # Reshape 3d tensor into a 2d one using `Flux.flatten`, at this point it should be (3, 3, 32, N)
    Flux.flatten,
    #Dropout(0.1),
    Dense(sz*sz*Nh, 2),
    softmax
    )
end

build_model (generic function with 1 method)

In [69]:
function build_reg_model(Nh,seqlen, delta)
    a = floor(Int8,Nh)
    sz = ceil(Int8,delta/8)
    
    println(delta,"-",sz)
    return Chain(
    x -> Flux.unsqueeze(x,4),
    # First convolution, operating upon a 28x28 image
    Conv((2, 2), seqlen=>a, pad=(1,1), relu),
    MaxPool((2,2)),

    # Second convolution, operating upon a 14x14 image
    Conv((2, 2), a=>Nh, pad=(1,1), relu),
    MaxPool((2,2)),

    # Third convolution, operating upon a 7x7 image
    Conv((2, 2), Nh=>Nh, pad=(1,1), relu),
    MaxPool((2,2)),

    # Reshape 3d tensor into a 2d one using `Flux.flatten`, at this point it should be (3, 3, 32, N)
    Flux.flatten,
    Dropout(0.1),
    (x->transpose(x)),
    GRU(1,Nh),
    GRU(Nh,Nh),
    (x -> x[:,end]),
    Dense(Nh, 1),
    (x -> x[1])
    )
end

build_reg_model (generic function with 1 method)

In [78]:
function accuracy(m,xtest,ytest)
    prediction = m.(xtest)
    len = length(ytest)
    m = zeros(2,2)
    for i in 1:len
        yh = Flux.onecold(prediction[i],[0,1])
        y = Flux.onecold(ytest[i],[0,1])
        a = Int(yh[1])+1
        b = y+1
        m[a,b] += 1
    end
    m = m ./ sum(m)
    return m
end

accuracy (generic function with 1 method)

In [79]:
function reg_accuracy(m,xtest,ytest)
    prediction = m.(xtest)
    len = length(ytest)
    m = zeros(2,2)
    for i in 1:len
        yh = prediction[i] > 0 ? 1 : 0
        y = ytest[i] > 0 ? 1 : 0
        a = Int(yh[1])+1
        b = y+1
        m[a,b] += 1
    end
    m = m ./ sum(m)
    return m
end

reg_accuracy (generic function with 1 method)

In [12]:
function plot_eval(m,xtest,ytest)
    prediction = m.(xtest)
    len = length(ytest)
    #println(len)
    pred = Float32[]
    actual = Float32[]
    total = 0
    for i in 1:len
        yh = Flux.onecold(prediction[i],[0,1])
        y = Flux.onecold(ytest[i],[0,1])
        append!(pred,yh)
        append!(actual,y)
    end
    plot(1:len,pred,size = (1000, 700),color="red",tickfontsize=28,guidefontsize=28,legendfontsize=28)
    plot!(1:len,actual,size = (1000, 700),color="blue",tickfontsize=28,guidefontsize=28,legendfontsize=28)
end

plot_eval (generic function with 1 method)

In [105]:
function myTrain(etf,seqlen,Nh,lr,mm,day0,day1, mlclass,i)
    #seqlen=7
    #mlclass = :clf
    (X,Y) = preprocessing(etf,day0,day1,mlclass)
    (xtrain,ytrain) = timeseriesDataset(X,Y,seqlen,mlclass)
    thd = 150
    xtest,ytest = xtrain[end-thd+1:end], ytrain[end-thd+1:end]
    xtrain,ytrain = xtrain[1:end-thd], ytrain[1:end-thd]
    batchsize = 20
    train_loader = Flux.Data.DataLoader(xtrain,ytrain, batchsize=batchsize,shuffle=false)
    #Nh=10
    delta = day1 - day0 + 1
    if mlclass == :reg
        m = build_reg_model(Nh,seqlen,delta)
    else
        m = build_model(Nh,seqlen,delta)
    end
    function loss(x, y)    
        yh = m.(x)
        #println(yh)
        #println(y)
        #println(Flux.Losses.logitcrossentropy(yh[1], y[1]))
        return sum(Flux.Losses.logitcrossentropy.(yh, y))
    end
    function acc_loss(x,y)
        (acc,y0,y1,y2) = accuracy(m,x,y)
        return (1-acc-y0+y1)*abs(y2-0.5)
    end
    function mse_loss(x,y)
        yh = m.(x)
        e = Flux.mae(yh,y)
        return e
    end
    evalcb() = @show loss(xtest,ytest)
    num_epoch = 25
    #lr = 0.01
    if mlclass == :reg
        @epochs num_epoch Flux.train!(mse_loss,Flux.params(m),train_loader,RMSProp(lr,mm))
        confmx = reg_accuracy(m,xtest,ytest)
        prediction = m.(xtest)
        println(size(prediction))
        #p = map(x -> x[end],prediction)
        #y = map(x -> x[end],ytest)
        #lp = getindex(size(prediction),1)
        #ly = getindex(size(y),1)
        plot(1:length(prediction),prediction,size = (1000, 700),color="red",tickfontsize=28,guidefontsize=28,legendfontsize=28)
        plot!(1:length(ytest),ytest,size = (1000, 700),color="blue",tickfontsize=28,guidefontsize=28,legendfontsize=28)
        dir="/Users/kahingleung/Downloads/gold-julia/"
        png("/Users/kahingleung/Downloads/gold-julia/plot-"*string(i)*".png")
    else
        @epochs num_epoch Flux.train!(loss,Flux.params(m),train_loader,RMSProp(lr,mm))
        confmx = accuracy(m,xtest,ytest)
    end
    println("accuracy:",(confmx[1,1]+confmx[2,2]))
    println(confmx[1,:])
    println(confmx[2,:])
    objcost = -1*(confmx[1,1]+confmx[2,2]-confmx[1,2]-confmx[2,1])
    return (m,objcost,xtest,ytest)
end
function myObjective(etf,seqlen,Nh,lr,mm,day0,day1,mlclass,i)
    (m,objcost,xtest,ytest) = myTrain(etf,seqlen,Nh,lr,mm,day0,day1,mlclass,i)
    Flux.reset!(m)
    return objcost
end

myObjective (generic function with 3 methods)

In [15]:
etf = getStock("2840.HK","3y")

,Open,Close,High,Low,Volume,ondate
,Float32,Float32,Float32,Float32,Int64,Date
1,985.5,990.0,990.0,985.5,4990,2018-01-19
2,988.0,987.0,989.0,986.0,13820,2018-01-22
3,990.0,991.5,992.5,990.0,14230,2018-01-23
4,994.0,996.5,997.5,994.0,6430,2018-01-24
5,1003.0,1010.0,1013.0,1003.0,52850,2018-01-25
6,1005.0,1005.0,1006.0,1002.0,15320,2018-01-26
7,1001.0,1000.0,1001.0,999.0,14080,2018-01-29
8,995.0,990.5,999.0,990.0,18110,2018-01-30
9,993.0,997.0,998.0,992.0,4790,2018-01-31


In [106]:
ho = @hyperopt for i=20,
            sampler = RandomSampler(), # This is default if none provided
            seqlen = StepRange(3, 5, 20),
            Nh = StepRange(5,3, 20),
            delta = StepRange(10,5, 25),
            #lr =  LinRange(1e-5,9e-5,10),
            lr =  exp10.(LinRange(-4,-3,10)),
            mm =  LinRange(0.75,0.95,5),
            day0 = StepRange(5,3, 10)
    println(i,"-",seqlen,"-",Nh,"-",lr,"-",mm,"-",day0,"-",delta)
   @show myObjective(etf,seqlen,Nh,lr,mm,day0,day0+delta,:reg,i)
end

1-3-5-0.0001668100537200059-0.85-8-10
[:PL, :return, :Close, Symbol("ORSI-8-8"), Symbol("ORSI-8-9")]
(702, 121)11
Float32[8.882732, 11.50449, 11.720596, 10.828548]
Float32[7.543152, 9.3754425, 11.852219, 12.168396]
Float32[12.502499, 14.789581, 15.368568, 15.373886]
Float32[8.882732, 11.50449, 11.720596, 10.828548]
Float32[7.543152, 9.3754425, 11.852219, 12.168396]
Float32[12.502499, 14.789581, 15.368568, 15.373886]
11-2
(150,)
accuracy:0.5466666666666666
[0.13333333333333333, 0.18]
[0.2733333333333333, 0.41333333333333333]
myObjective(etf, seqlen, Nh, lr, mm, day0, day0 + delta, :reg, i) = -0.09333333333333332
2-13-5-0.00021544346900318823-0.8-8-25
[:PL, :return, :Close, Symbol("ORSI-8-8"), Symbol("ORSI-8-9")]
(672, 676)26
Float32[-22.039062, -23.175922, -24.400112, -25.917557]
Float32[-16.465065, -20.631721, -21.912792, -23.247032]
Float32[-17.519463, -19.146164, -20.39457, -22.469883]
Float32[-22.039062, -23.175922, -24.400112, -25.917557]
Float32[-16.465065, -20.631721, -21.912792,

┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

[:PL, :return, :Close, Symbol("ORSI-8-8"), Symbol("ORSI-8-9")]
(672, 676)26
Float32[-22.039062, -23.175922, -24.400112, -25.917557]
Float32[-16.465065, -20.631721, -21.912792, -23.247032]
Float32[-17.519463, -19.146164, -20.39457, -22.469883]
Float32[-22.039062, -23.175922, -24.400112, -25.917557]
Float32[-16.465065, -20.631721, -21.912792, -23.247032]
Float32[-17.519463, -19.146164, -20.39457, -22.469883]
26-4
(150,)
accuracy:0.7666666666666666
[0.26666666666666666, 0.09333333333333334]
[0.14, 0.5]
myObjective(etf, seqlen, Nh, lr, mm, day0, day0 + delta, :reg, i) = -0.5333333333333332
6-8-5-0.0001291549665014884-0.8-5-10
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(708, 121)11
Float32[-4.718628, -4.2850075, -5.191963, -5.8704376]
Float32[-3.3991623, -4.4669266, -4.2228813, -5.1044426]
Float32[-8.919769, -7.9227676, -10.5271225, -11.731354]
Float32[-4.718628, -4.2850075, -5.191963, -5.8704376]
Float32[-3.3991623, -4.4669266, -4.2228813, -5.1044426]
Float32[-8.919769,

┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 12
└ @ Main /U

(150,)
accuracy:0.56
[0.23333333333333334, 0.26666666666666666]
[0.17333333333333334, 0.32666666666666666]
myObjective(etf, seqlen, Nh, lr, mm, day0, day0 + delta, :reg, i) = -0.12000000000000005
10-3-8-0.0001291549665014884-0.8999999999999999-5-20
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(688, 441)21
Float32[-16.984947, -23.183212, -24.460663, -23.277512]
Float32[-22.599937, -31.289436, -37.49622, -39.395164]
Float32[-19.349712, -26.876652, -29.43251, -29.776062]
Float32[-16.984947, -23.183212, -24.460663, -23.277512]
Float32[-22.599937, -31.289436, -37.49622, -39.395164]
Float32[-19.349712, -26.876652, -29.43251, -29.776062]
21-3
(150,)
accuracy:0.56
[0.26666666666666666, 0.3]
[0.14, 0.29333333333333333]
myObjective(etf, seqlen, Nh, lr, mm, day0, day0 + delta, :reg, i) = -0.12000000000000005
11-8-8-0.0002782559402207126-0.85-5-10
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(708, 121)11
Float32[-4.718628, -4.2850075, -5.191963, -5.8704376]
Floa

...

(150,)
accuracy:0.76
[0.26, 0.09333333333333334]
[0.14666666666666667, 0.5]
myObjective(etf, seqlen, Nh, lr, mm, day0, day0 + delta, :reg, i) = -0.52
14-3-11-0.001-0.95-8-25
[:PL, :return, :Close, Symbol("ORSI-8-8"), Symbol("ORSI-8-9")]
(672, 676)26
Float32[-22.039062, -23.175922, -24.400112, -25.917557]
Float32[-16.465065, -20.631721, -21.912792, -23.247032]
Float32[-17.519463, -19.146164, -20.39457, -22.469883]
Float32[-22.039062, -23.175922, -24.400112, -25.917557]
Float32[-16.465065, -20.631721, -21.912792, -23.247032]
Float32[-17.519463, -19.146164, -20.39457, -22.469883]
26-4
(150,)
accuracy:0.7
[0.23333333333333334, 0.12666666666666668]
[0.17333333333333334, 0.4666666666666667]
myObjective(etf, seqlen, Nh, lr, mm, day0, day0 + delta, :reg, i) = -0.3999999999999999
15-18-11-0.0001-0.8-5-25
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(678, 676)26
Float32[-5.819477, -5.43919, -4.657501, -4.841305]
Float32[-9.396957, -11.929012, -11.943882, -11.508736]
Float32[-23

┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 12
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 13
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 14
└ @ Main 

(150,)
accuracy:0.72
[0.22, 0.09333333333333334]
[0.18666666666666668, 0.5]
myObjective(etf, seqlen, Nh, lr, mm, day0, day0 + delta, :reg, i) = -0.43999999999999995
18-8-14-0.0001-0.85-8-25
[:PL, :return, :Close, Symbol("ORSI-8-8"), Symbol("ORSI-8-9")]
(672, 676)26
Float32[-22.039062, -23.175922, -24.400112, -25.917557]
Float32[-16.465065, -20.631721, -21.912792, -23.247032]
Float32[-17.519463, -19.146164, -20.39457, -22.469883]
Float32[-22.039062, -23.175922, -24.400112, -25.917557]
Float32[-16.465065, -20.631721, -21.912792, -23.247032]
Float32[-17.519463, -19.146164, -20.39457, -22.469883]
26-4
(150,)
accuracy:0.58
[0.12666666666666668, 0.14]
[0.28, 0.4533333333333333]
myObjective(etf, seqlen, Nh, lr, mm, day0, day0 + delta, :reg, i) = -0.15999999999999992
19-8-8-0.0005994842503189409-0.75-5-15
[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(698, 256)16
Float32[-13.412476, -13.435123, -15.05933, -18.486206]
Float32[4.951828, 3.7055435, 3.7920036, 2.4498253]
Float32[1

┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 12
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 13
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 14
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 15
└ @ Main

Hyperoptimizer(20, (:seqlen, :Nh, :delta, :lr, :mm, :day0), (3:5:18, 5:3:20, 10:5:25, [0.0001, 0.0001291549665014884, 0.0001668100537200059, 0.00021544346900318823, 0.0002782559402207126, 0.00035938136638046257, 0.00046415888336127773, 0.0005994842503189409, 0.000774263682681127, 0.001], range(0.75, stop=0.95, length=5), 5:3:8), Any[Real[3, 5, 10, 0.0001668100537200059, 0.85, 8], Real[13, 5, 25, 0.00021544346900318823, 0.8, 8], Real[3, 8, 20, 0.0005994842503189409, 0.8999999999999999, 5], Real[18, 20, 15, 0.0001291549665014884, 0.75, 5], Real[3, 17, 25, 0.00046415888336127773, 0.8999999999999999, 8], Real[8, 5, 10, 0.0001291549665014884, 0.8, 5], Real[13, 14, 20, 0.0001668100537200059, 0.8, 8], Real[13, 5, 25, 0.001, 0.95, 8], Real[8, 5, 10, 0.0001668100537200059, 0.8999999999999999, 5], Real[3, 8, 20, 0.0001291549665014884, 0.8999999999999999, 5], Real[8, 8, 10, 0.0002782559402207126, 0.85, 5], Real[13, 17, 25, 0.0001668100537200059, 0.75, 8], Real[8, 20, 15, 0.0005994842503189409, 0.

In [107]:
best_params, min_f = ho.minimizer, ho.minimum

(Real[8, 8, 15, 0.0005994842503189409, 0.75, 5], -0.5466666666666667)

In [108]:
seqlen=best_params[1]
Nh=best_params[2]
delta=best_params[3]
lr=best_params[4]
mm=best_params[5]
day0=best_params[6]
(m,acc,xtest,ytest) = myTrain(etf,seqlen,Nh,lr,mm,day0,day0+delta,:reg,21)

[:PL, :return, :Close, Symbol("ORSI-5-5"), Symbol("ORSI-5-6")]
(698, 256)16
Float32[-13.412476, -13.435123, -15.05933, -18.486206]
Float32[4.951828, 3.7055435, 3.7920036, 2.4498253]
Float32[1.8990479, 2.1723785, 0.109809875, -1.6851349]
Float32[-13.412476, -13.435123, -15.05933, -18.486206]
Float32[4.951828, 3.7055435, 3.7920036, 2.4498253]
Float32[1.8990479, 2.1723785, 0.109809875, -1.6851349]
16-2
(150,)
accuracy:0.7066666666666667
[0.26666666666666666, 0.15333333333333332]
[0.14, 0.44]


┌ Info: Epoch 1
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 2
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 3
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 4
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 5
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 6
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 7
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 8
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 9
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 10
└ @ Main /Users/kahingleung/.julia/packages/Flux/sY3yx/src/optimise/train.jl:136
┌ Info: Epoch 11
└ @ Main /Us

(Chain(#56, Conv((2, 2), 8=>8, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((2, 2), 8=>8, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((2, 2), 8=>8, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dropout(0.1), #57, Recur(GRUCell(1, 8)), Recur(GRUCell(8, 8)), #58, Dense(8, 1), #59), -0.41333333333333333, Array{Float32,3}[[-0.8122215 -11.206036 … 0.5885658 1.437458; -4.566025 -15.030685 … -0.32451248 0.97781754; … ; -14.182163 -21.853523 … -3.7311974 -2.972416; -14.520603 -21.716015 … -3.9956055 -3.2219963]

[-3.0699806 2.9696312 … 0.95451355 2.046238; -6.3208847 -1.4033012 … 0.48621368 0.7576027; … ; -20.659424 -11.29324 … -3.0496979 -3.0388947; -19.907253 -11.714874 … -3.549492 -3.3257103]

[20.377148 11.060299 … 4.3319817 3.9017715; 21.357159 9.559814 … 4.7958794 3.9444656; … ; 5.9752235 -3.7168732 … 2.1596184 0.09624863; 7.637188 -2.6515388 … 1.6633072 -0.33418274]

[13.879765 11.218296 … 1.1486664 2.8191605; 11.709358 